In [10]:
import zipfile
import os
from pathlib import Path
import pprint
import pandas as pd

In [11]:
def unzip_given_file_to_folder(file_path, folder):
    with zipfile.ZipFile(file_path, 'r') as z:
        z.extractall(folder)

In [12]:
unzipped_data = '/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/unzipped_data'

In [13]:
os.chdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/zipped_data')
for file in os.listdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/zipped_data'):
    if file.endswith('.zip'):
        file_path = os.path.abspath(file)
        unzip_given_file_to_folder(file_path=file_path, folder=unzipped_data)
    else:
        print(f"{file} - Not a ZIP file!")

In [14]:
list_csv_files = os.listdir(unzipped_data)
print(len(list_csv_files))
list_csv_files

229


['41559__Temperature__D.csv',
 '41559__Temperature__H.csv',
 '41559__Temperature__unknown.csv',
 '41559__Wind__all_data.csv',
 '41559__Wind__D.csv',
 '41559__Wind__H.csv',
 '8567__Rain__all_data.csv',
 '8567__Rain__D.csv',
 '8567__Rain__M.csv',
 '8567__Rain__S.csv',
 '8567__Screen_Observations__A.csv',
 '8567__Screen_Observations__D.csv',
 '8567__Screen_Observations__M.csv',
 '8567__Screen_Observations__S.csv',
 '8567__Temperature__A.csv',
 '8567__Temperature__all_data.csv',
 '8567__Temperature__D.csv',
 '8567__Temperature__unknown.csv',
 '8567__Wind__all_data.csv',
 '8567__Wind__D.csv',
 '8567__Wind__M.csv',
 '8567__Wind__S.csv',
 '111__RainD.csv',
 '111__Screen_ObservationsD.csv',
 '111__TemperatureD.csv',
 '111__WindD.csv',
 '12442__Rain__all_data.csv',
 '12442__Rain__D.csv',
 '12442__Rain__H.csv',
 '12442__Rain__Q.csv',
 '12442__Rain__R.csv',
 '12442__Screen_Observations__H.csv',
 '12442__Temperature__all_data.csv',
 '12442__Temperature__D.csv',
 '12442__Temperature__H.csv',
 '1244

# Filling each station with climatic parameters

In [20]:
df_stations = pd.read_csv('stations.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'stations.csv'

In [ ]:
columns = ['station_id', 'Observation time UTC', 'Temperature', 'Screen_Observations', 'Rain', 'Wind']
dict_stations = {}
df = pd.DataFrame(columns=columns)
df['Observation time UTC'] = pd.to_datetime(df['Observation time UTC']).dt.date
dict_stations = {station_id: df for station_id in df_stations['StationID']}
print(sorted(dict_stations))

[2592, 2685, 3145, 3445, 8567, 12442, 18234, 21938, 25354, 25531, 31857, 40750, 40984, 41212, 41229, 41559]


In [ ]:
def get_column_df_from_csv(csv_file_path: Path, column: str) -> pd.DataFrame | None: # temp df
    column_df = pd.read_csv(csv_file_path)
    column_df['Observation time UTC'] = pd.to_datetime(column_df['Observation time UTC']).dt.date
    if column == 'Temperature' or column == 'Rain':
        column_df.drop(column_df[column_df['Frequency [D/H/S]'] != 'Daily'].index, inplace=True)
    elif column == 'Wind':
        column_df.drop(column_df[column_df['Frequency [D/H]'] != 'Daily'].index, inplace=True)  
    
    return column_df

In [ ]:
def merge_station_df_with_column_df(station_df: pd.DataFrame, column_df: pd.DataFrame, column: str) -> pd.DataFrame:
    column_to_columns = {
    'Temperature': 'Mean Temperature [Deg C]',
    'Rain': 'Rainfall [mm]',
    'Wind': 'Speed [m/s]',
    'Screen_Observations': 'Relative humidity [%]'
    }

    station_df = pd.merge(station_df, column_df[['Observation time UTC', column_to_columns[column]]], on='Observation time UTC', how='outer')
    station_df[column] = station_df[column_to_columns[column]]
    station_df = station_df[columns].sort_values('Observation time UTC', ascending=False)

    return station_df

### Filling parameters with daily data

In [ ]:
for file_name in list_csv_files:
    try:
        column, param = file_name.split('__')[1:3]
    except:
        continue

    station_id: int = int(file_name.split('__')[0])

    if param.startswith('D.csv'):   # Daily data
        print(f"station_id: {station_id}, column: {column}, param:{param}")
       
        csv_file_path = Path(unzipped_data + file_name)
        column_df = get_column_df_from_csv(csv_file_path=csv_file_path, column=column)

        station_df = dict_stations[station_id]
        
        merged_station_df = merge_station_df_with_column_df(station_df=station_df, column_df=column_df, column=column)
        merged_station_df['station_id'] = merged_station_df['station_id'].fillna(station_id)
        dict_stations[station_id] = merged_station_df

print(dict_stations[40750])

NameError: name 'list_csv_files' is not defined

In [ ]:
for id in dict_stations:
    path_filled_stations = Path('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/stations_filled/'+str(id)+'.csv')
    dict_stations[id].to_csv(path_filled_stations)

print(os.listdir('/media/lenalexu/64522E74522E4ADE/D/Whitireia/ARP/data/stations_filled_daily/'))